# Part 1: Stock data from Alpha Vantage

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
import time

def request_stock_price_hist(symbol, token, sample = False):
    """
    This function helps the user to retrieve historical stock prices for the
    specified symbol from Alpha Vantage.

    Parameters
    ----------
    symbol : String
        Stock symbol, e.g. Apple is AAPL.
    token : String
        Register an account on alphavantage.co and get your API.
    sample : Boolean, optional
        Set to True to take a sample of the data only.

    Returns
    -------
    df : Pandas DataFrame
        A Pandas DataFrame containing stock price information.

    """
    if sample == False:
        q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}'
    else:
        q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}'
    
    print("Retrieving stock price data from Alpha Vantage (This may take a while)...")
    r = requests.get(q_string.format(symbol, token))
    print("Data has been successfully downloaded...")
    date = []
    colnames = list(range(0, 7))
    df = pd.DataFrame(columns = colnames)
    print("Sorting the retrieved data into a dataframe...")
    for i in tqdm(r.json()['Time Series (Daily)'].keys()):
        date.append(i)
        row = pd.DataFrame.from_dict(r.json()['Time Series (Daily)'][i], orient='index').reset_index().T[1:]
        df = pd.concat([df, row], ignore_index=True)
    df.columns = ["open", "high", "low", "close", "adjusted close", "volume", "dividend amount", "split cf"]
    df['date'] = date
    return df

def request_quote(symbol, token):
    """
    This function helps the user to retrieve current stock quote for the
    specified symbol from Alpha Vantage.

    Parameters
    ----------
    symbol : String
        Stock symbol, e.g. Apple is AAPL.
    token : String
        Register an account on alphavantage.co and get your API.

    Returns
    -------
    df : Pandas DataFrame
        A Pandas DataFrame containing stock price information.

    """
    q_string = 'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={}&apikey={}'
    r = requests.get(q_string.format(symbol, token))
    colnames = [x.split('. ')[1] for x in list(r.json()['Global Quote'].keys())]
    df = pd.DataFrame.from_dict(r.json()['Global Quote'], orient='index').reset_index().T[1:]
    df.columns = colnames
    return df

def request_symbols(token):
    """
    This function helps the user to retrieve all currently active listed 
    stocks and their symbols from Alpha Vantage.

    Parameters
    ----------
    token : String
        Register an account on alphavantage.co and get your API.

    Returns
    -------
    df : Pandas DataFrame
        A Pandas DataFrame containing stock price information.

    """
    q_string = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={}'   
    print("Retrieving stock symbols from Alpha Vantage...")
    r = requests.get(q_string.format(token)).content
    print("Data has been successfully downloaded...")
    r = r.decode("utf-8")
    colnames = list(range(0, 6))
    df = pd.DataFrame(columns = colnames)
    print("Sorting the retrieved data into a dataframe...")
    for i in tqdm(range(1, len(r.split('\r\n'))-1)):
        row = pd.DataFrame(r.split('\r\n')[i].split(',')).T
        df = pd.concat([df, row], ignore_index=True)
    df.columns = r.split('\r\n')[0].split(',')
    return df

def save_stock_price_hist(symbol_list, token, pwd=''):
    """
    This function helps the user to download all historical stock prices
    for each symbol on the symbol_list parameter from Alpha Vantage. There 
    will be a CSV file for each symbol and the user can specify the output 
    path for the CSV files.
    
    Please note that Alpha Vantage does not accept more than 5 calls per
    minute. Therefore if your symbol_list object is long, the function will
    automatically set waiting time to ensure there is no break once the
    function is set to run.

    Parameters
    ----------
    symbol_list : List
        A list of symbols (string type) to query.  
    token : String
        Register an account on alphavantage.co and get your API.
    pwd : String, optional
        The path to the directory where the CSV files will be saved.

    Returns
    -------
    None

    """
    n = datetime.now()
    curr = 0
    for i in symbol_list:
        ind = symb.index(i)
        l = datetime.now()
        if (l - n < timedelta(minutes=1)) and (ind-curr == 5):
            print("We made 5 API calls in the last minute, taking a break...")
            time.sleep((timedelta(minutes=2) - (l - n)).seconds)
            curr = i
        else:
            df = request_stock_price_hist(i, token)
            df.to_csv('{}/{}_{}.csv'.format(pwd, i, datetime.today().strftime('%Y%m%d')))
    return None

In [2]:
alphavantage_token = 'use your own token'

# Extract a full list of symbols from Alpha Vantage
sym_df = request_symbols(alphavantage_token)

# Grab only NASDAQ symbols from the extracted dataframe
symb = list(sym_df.loc[sym_df['exchange'] == 'NASDAQ']['symbol'])

# Save NASDAQ stock historical prices into Data folder (sample only)
save_stock_price_hist(symb[0:5], alphavantage_token, 'Data')

Retrieving stock symbols from Alpha Vantage...


  0%|▎                                                                              | 34/9353 [00:00<00:27, 336.83it/s]

Data has been successfully downloaded...
Sorting the retrieved data into a dataframe...


100%|█████████████████████████████████████████████████████████████████████████████| 9353/9353 [00:28<00:00, 331.29it/s]


Retrieving stock price data from Alpha Vantage (This may take a while)...


...


100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 446.67it/s]

Data has been successfully downloaded...
Sorting the retrieved data into a dataframe...
Retrieving stock price data from Alpha Vantage (This may take a while)...



  0%|▏                                                                                | 9/3786 [00:00<00:46, 81.12it/s]

Data has been successfully downloaded...
Sorting the retrieved data into a dataframe...


100%|██████████████████████████████████████████████████████████████████████████████| 3786/3786 [00:49<00:00, 76.46it/s]


# Part 2: News articles from Contextual Websearch API

In [3]:
import requests
from textblob import TextBlob
import pandas as pd
from datetime import timedelta
from datetime import datetime

def sentiment(text):
    """
    Your input is a string. The function will invoke the TextBlobk package's
    sentiment function and returns values of polarity and subjectivity to user.

    Parameters
    ----------
    text : A string.

    Returns
    -------
    A textblob.en.sentiments.Sentiment object.

    """
    return TextBlob(text).sentiment

def request_news(start_date, end_date, topic, token):
    """
    This function helps the user to extract news articles from a given period
    regarding the topic through Contextual Web Search API. 

    Parameters
    ----------
    start_date : String
        The start date of the period of interest. e.g. 2020-01-01
    end_date : String
        The end date of the period of interest. e.g. 2020-01-31
    topic : String
        The topic of interest. e.g. NASDAQ
    token : String
        Your Contextual Web Search token on RapidAPI.

    Returns
    -------
    big_df : Pandas DataFrame
        A Pandas DataFrame containing news articles within the defined window 
        of interest.

    """
    delt = (datetime.strptime(end_date, '%Y-%m-%d').date() - datetime.strptime(start_date, '%Y-%m-%d').date()).days
    date_list = [(str((datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=x)).date())) for x in range(0, delt, 7)]
    
    big_df = pd.DataFrame()
    
    for j in range(0, len(date_list)-1):
        start_d = date_list[j]
        end_d = date_list[j+1]
    
        url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"
        querystring = {"fromPublishedDate":start_d,"toPublishedDate":end_d,"autoCorrect":"false","pageNumber":"1","pageSize":"50","q":topic,"safeSearch":"false"}
        headers = {
            'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
            'x-rapidapi-key': token
        }
        r = requests.request("GET", url, headers=headers, params=querystring)
        while r.status_code != 200:
            print("API call was not successful, trying again in 10 seconds...")
            time.sleep(10)
            r = requests.request("GET", url, headers=headers, params=querystring)
        colnames = list(range(0, 11))
        df = pd.DataFrame(columns = colnames)
        no_news = len(r.json()['value'])
        if no_news > 0:
            print("Successfully retrieved news articles between {} and {}".format(start_d, end_d))
            for i in range(0, no_news):
                row = pd.DataFrame(pd.DataFrame.from_dict(r.json()['value'][i], orient='index').reset_index().T.iloc[1])
                df = pd.concat([df, row.T], ignore_index=True)
            
            df.columns = list(pd.DataFrame.from_dict(r.json()['value'][0], orient='index').reset_index().T.iloc[0])
            df = df[['title', 'url', 'description', 'body', 'datePublished']]
            
            big_df = pd.concat([big_df, df], ignore_index=True)
        else:
            print("There were no news articles between {} and {}".format(start_d, end_d))
            pass
    
    big_df['dates'] = [x[0:10] for x in big_df['datePublished']]
    del(big_df['datePublished'])
    
    return big_df

# Part 3: Compile a training dataset with both stock prices and news at daily level

In [4]:
from glob import glob
from datetime import timedelta
from datetime import datetime

def compile_data(symbols, period_start, period_end, pwd):
    """
    This function helps user to compile a dataset of stock prices.

    Parameters
    ----------
    symbols : List
        A list of symbols that represent the stocks of interest.
    period_start : String
        Period start date of the output table.
    period_end : TYPE
        Period end date of the output table.
    pwd : String
        Directory path where the stock price CSV files are stored.

    Returns
    -------
    df : Pandas DataFrame
        A Pandas DataFrame containing stock price information within the
        defined window of interest.

    """
    df = pd.DataFrame()
    
    dates = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(period_start, period_end))]
    df['dates'] = dates
    for i in symbols:
        temp = pd.read_csv(glob('{}{}*.csv'.format(pwd, i))[0], index_col=0)[['date','adjusted close']]
        temp.columns = ['dates', i]
        df = df.merge(temp, on='dates', how='left')
        
    df.columns = ['dates'] + symbols
    return df

In [5]:
# First define a list of stocks we want (sample only)
symb_list = ['FB', 'NFLX', 'GOOGL', 'MSFT', 'AMZN', 'ADBE']

# Put together a dataframe with all stock prices from Data folder
stocks = compile_data(symb_list, '2010-01-01', '2020-08-28', 'Data/')

# Show the dataframe (just stock prices at this stage)
stocks.head()

,dates,FB,NFLX,GOOGL,MSFT,AMZN,ADBE
0,2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,NaN,7.6400,314.3454,24.1710,133.90,37.09
4,2010-01-05,NaN,7.3586,312.9611,24.1788,134.69,37.70


In [ ]:
rapidapi_contextualweb_token = 'use your own token'

# Extract news about NASDAQ
news = request_news('2020-01-01', '2020-5-31', 'NASDAQ', rapidapi_contextualweb_token)
news['positivity'] = [sentiment(x)[0] for x in news['body']]
news['subjectivity'] = [sentiment(x)[1] for x in news['body']]
news['tally'] = 1

# Aggregate news at daily level
agg_news = pd.DataFrame(news.groupby('dates').agg(avg_pos=pd.NamedAgg(column='positivity', aggfunc=np.mean),
                                       avg_subj=pd.NamedAgg(column='subjectivity', aggfunc=np.mean),
                                       news_cnt=pd.NamedAgg(column='tally', aggfunc=sum))).reset_index(level=0)

# Join news to stock price history data
stocks = stocks.merge(agg_news[['dates', 'avg_pos', 'avg_subj', 'news_cnt']], on='dates', how='left')

In [9]:
output = pd.read_csv("Sample_Output/nasdaq_sample.csv", index_col = 0)
output.tail(20)

,dates,FB,NFLX,GOOGL,MSFT,AMZN,ADBE,avg_pos,avg_subj,news_cnt
3873,2020-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3874,2020-08-10,18.3237,483.38,1496.82,207.7448,3148.16,443.29,0.086875,0.313958,2.0
3875,2020-08-11,18.5019,466.93,1480.54,202.8866,3080.67,435.23,0.148692,0.401033,4.0
3876,2020-08-12,18.4821,475.47,1507.24,208.6825,3162.24,445.36,-0.082602,0.205409,1.0
3877,2020-08-13,18.5128,481.33,1516.65,208.1937,3161.02,450.02,0.079927,0.315074,12.0
3878,2020-08-14,18.5296,482.68,1504.63,208.3932,3148.02,447.60,0.014830,0.246745,31.0
3879,2020-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3880,2020-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3881,2020-08-17,18.6603,482.35,1516.24,209.7698,3182.41,451.58,-0.187074,0.510938,2.0
3882,2020-08-18,18.9078,491.87,1555.78,210.9769,3312.49,463.51,0.117609,0.472896,1.0
